In [2]:
import pandas as pd
import numpy as np
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
data = pd.read_csv('train.csv')

/Library/Python/2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (8,9,10,11,12,43,157,196,214,225,228,229,231,235,238) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [4]:
print(data['target'].sum())
print(len(data.index))
data.head()

33773
145231


,ID,VAR_0001,VAR_0002,VAR_0003,VAR_0004,VAR_0005,VAR_0006,VAR_0007,VAR_0008,VAR_0009,...,VAR_1926,VAR_1927,VAR_1928,VAR_1929,VAR_1930,VAR_1931,VAR_1932,VAR_1933,VAR_1934,target
0,2,H,224,0,4300,C,0,0,False,False,...,98,98,998,999999998,998,998,9998,9998,IAPS,0
1,4,H,7,53,4448,B,1,0,False,False,...,98,98,998,999999998,998,998,9998,9998,IAPS,0
2,5,H,116,3,3464,C,0,0,False,False,...,98,98,998,999999998,998,998,9998,9998,IAPS,0
3,7,H,240,300,3200,C,0,0,False,False,...,98,98,998,999999998,998,998,9998,9998,RCC,0
4,8,R,72,261,2000,N,0,0,False,False,...,98,98,998,999999998,998,998,9998,9998,BRANCH,1


In [6]:
f = data.drop([data.columns[0]], axis=1)._get_numeric_data()

In [7]:
from sklearn import tree
from sklearn.cross_validation import KFold

kf = KFold(len(f.index), n_folds=5)

for train, test in kf:
    print("train:%s test:%s" % (train, test))
    features = f.irow(train)
    samples = features['target']
    features = features.drop('target', axis=1).fillna(0)    
    clf = tree.DecisionTreeClassifier()
    print("Fitting %d samples..." % len(train))
    clf.fit(features, samples)
    
    print("Scoring...")
    score = clf.score(f.irow(test).drop('target', axis=1).fillna(0), f.irow(test)['target'])  
    print("Score %f" % score)


train:[ 29047  29048  29049 ..., 145228 145229 145230] test:[    0     1     2 ..., 29044 29045 29046]
Fitting 116184 samples...
Scoring...
Score 0.700485
train:[     0      1      2 ..., 145228 145229 145230] test:[29047 29048 29049 ..., 58090 58091 58092]
Fitting 116185 samples...
Scoring...
Score 0.700923
train:[     0      1      2 ..., 145228 145229 145230] test:[58093 58094 58095 ..., 87136 87137 87138]
Fitting 116185 samples...
Scoring...
Score 0.699718
train:[     0      1      2 ..., 145228 145229 145230] test:[ 87139  87140  87141 ..., 116182 116183 116184]
Fitting 116185 samples...
Scoring...
Score 0.702300
train:[     0      1      2 ..., 116182 116183 116184] test:[116185 116186 116187 ..., 145228 145229 145230]
Fitting 116185 samples...
Scoring...
Score 0.699305


In [6]:
test = pd.read_csv('test.csv')

/Library/Python/2.7/site-packages/pandas/io/parsers.py:1170: DtypeWarning: Columns (8,9,10,11,12,43,157,167,177,196,214,225,228,229,231,235,238) have mixed types. Specify dtype option on import or set low_memory=False.
  data = self._reader.read(nrows)


In [7]:
test_features = test[f.drop('target',axis=1).columns].fillna(0)
test['target'] = pd.Series(clf.predict(test_features))

In [8]:
test[['ID', 'target']].astype(np.int).to_csv('submission.csv', index=False)